# Análisis Exploratorio de Datos - Mercado Inmobiliario

Este cuaderno realiza la limpieza y análisis exploratorio de los datos scrapeados de ZonaProp.

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import ast

# Configuración de estilo
sns.set_theme(style="whitegrid")
pd.set_option('display.max_columns', None)

## 1. Carga de Datos

In [59]:
# Leemos el dataset
try:
    df = pd.read_csv("../data/raw/real_estate_listings.csv")
    print(f"Dimensiones iniciales: {df.shape}")
    display(df.head())
except FileNotFoundError:
    print("Archivo no encontrado. Verifica la ruta.")

Dimensiones iniciales: (60, 8)


,price,location,features,expensas,disposition,seniority,description,url
0,USD 76.000,"Floresta, Capital Federal","['44 m² tot.', '1 amb.', '1 baño']",NaN,Frente,NaN,Departamento de 1 ambiente a estrenar - Pronta...,https://www.zonaprop.com.ar/propiedades/clasif...
1,Consultar precio,"Balvanera, Capital Federal","['80 m² tot.', '3 amb.', '2 dorm.', '2 baños']",NaN,NaN,NaN,- Unidad de tres ambientes con dos baños compl...,https://www.zonaprop.com.ar/propiedades/clasif...
2,USD 300.000,"Recoleta, Capital Federal","['130 m² tot.', '4 amb.', '3 dorm.', '3 baños'...",$ 520.000 Expensas,Frente,NaN,Paraná y Arenales – A metros de Plaza Vicente ...,https://www.zonaprop.com.ar/propiedades/clasif...
3,USD 150.000,"Villa Pueyrredón, Capital Federal","['2 un.', '3 amb.', '58 a 68 m² tot.', 'Parril...",NaN,Frente,NaN,Excelentes semipisos de 3 y 4 ambientes con co...,https://www.zonaprop.com.ar/propiedades/empren...
4,USD 135.000,"Palermo, Capital Federal","['54 m² tot.', '2 amb.', '1 dorm.', '1 baño']",NaN,Frente,NaN,_ Departamento de un dormitorio en venta ubica...,https://www.zonaprop.com.ar/propiedades/clasif...


## 2. Limpieza y Procesamiento de Datos

In [ ]:
# 1. Limpieza de la columna features
# Quitamos los corchetes y comillas para trabajar con un string separado por comas
df['features_clean'] = df['features'].str.replace(r"[\[\]']", "", regex=True)

# 2. Función optimizada con Regex 
def extract_opt(text, pattern):
    match = re.search(fr'(\d+)\s*(?:{pattern})', text, re.IGNORECASE)
    return int(match.group(1)) if match else 0

# 3. Extracción de features 
df['m2'] = df['features_clean'].apply(lambda x: extract_opt(x, 'm²|metros'))
df['ambientes'] = df['features_clean'].apply(lambda x: extract_opt(x, 'amb'))
df['dormitorios'] = df['features_clean'].apply(lambda x: extract_opt(x, 'dorm'))
df['baños'] = df['features_clean'].apply(lambda x: extract_opt(x, 'baño|bano'))
df['cocheras'] = df['features_clean'].apply(lambda x: extract_opt(x, 'coch'))

# 4. Limpieza de precio 
df['precio_usd'] = pd.to_numeric(df['price'].str.replace(r'[^0-9]', '', regex=True), errors='coerce')

# 5. Filtrado y Limpieza Final
df_clean = df.dropna(subset=['precio_usd']).copy()
df_clean['ubicación'] = df_clean['location']

# Definimos columnas a mantener 
cols_finales = ['precio_usd', 'ubicación', 'm2', 'ambientes', 'dormitorios', 'baños', 'cocheras']
df_clean = df_clean[cols_finales].fillna(0)

df_clean.head()

,precio_usd,ubicación,m2,ambientes,dormitorios,baños,cocheras
0,117777.00,"Caballito, Capital Federal",31,1,0,1,0
1,224000.00,"Palermo Soho, Palermo",113,1,1,1,0
2,255000.00,"Recoleta, Capital Federal",108,4,3,2,1
3,133984.00,"Belgrano, Capital Federal",152,3,0,0,0
4,2100000.00,"Puerto Madero, Capital Federal",260,4,3,4,2


## 3. EDA